# Codex Analysis

In [1]:
import numpy as np
import pandas as pd

## read_csv()
We start off by what many consider one of the best examples of high quality API. The `read_csv()` function opened a whole new realm of possibility for the data analyst, all those Excel based workflows had now a natural evolution in the form of Pandas Dataframes.

In [2]:
df = pd.read_csv("codex.csv")

In [3]:
df.axes

[RangeIndex(start=0, stop=2143214, step=1),
 Index(['question_id', 'title', 'tags', 'creation_date', 'accepted_answer',
        'favorite_count', 'answer_count', 'comment_count', 'view_count'],
       dtype='object')]

In [4]:
df.head()

,question_id,title,tags,creation_date,accepted_answer,favorite_count,answer_count,comment_count,view_count
0,44919655,How to add an onclick method on dynamically cr...,javascript|jquery,2017-07-05 07:29:18.98 UTC,NaN,NaN,5,11,52
1,45962870,Html clone in Javascript,javascript|jquery|html,2017-08-30 14:16:55.68 UTC,45964230.0,2.0,5,11,106
2,47409716,'number' is not a valid type for an input tag ...,asp.net|html5|iis|webforms,2017-11-21 09:36:17.393 UTC,47515741.0,NaN,7,14,227
3,42415666,Error:Cause: unable to find valid certificatio...,android|android-studio,2017-02-23 12:22:43.983 UTC,NaN,1.0,7,12,17553
4,45001657,Codeigniter User's Data,codeigniter|view|details,2017-07-09 22:29:15.42 UTC,NaN,0.0,7,15,721


By exploring the head of the Dataframe we can quickly see that if we do not have a column telling us whether our question was answered or not, what we have is the information encoded in the **accepted_answer** column. That number clearly has a similarities with the **question_id**, we can deduce that if a question has an answer, its answer id will be stored, otherwise we will found a NaN.

In [5]:
columns_to_clean = ["accepted_answer", "favorite_count"]
polished_columns =  df[columns_to_clean].fillna(0).astype(int)
df[columns_to_clean] = polished_columns
df.head()

,question_id,title,tags,creation_date,accepted_answer,favorite_count,answer_count,comment_count,view_count
0,44919655,How to add an onclick method on dynamically cr...,javascript|jquery,2017-07-05 07:29:18.98 UTC,0,0,5,11,52
1,45962870,Html clone in Javascript,javascript|jquery|html,2017-08-30 14:16:55.68 UTC,45964230,2,5,11,106
2,47409716,'number' is not a valid type for an input tag ...,asp.net|html5|iis|webforms,2017-11-21 09:36:17.393 UTC,47515741,0,7,14,227
3,42415666,Error:Cause: unable to find valid certificatio...,android|android-studio,2017-02-23 12:22:43.983 UTC,0,1,7,12,17553
4,45001657,Codeigniter User's Data,codeigniter|view|details,2017-07-09 22:29:15.42 UTC,0,0,7,15,721


## The ~
The `~` simply let us take the inverse of an array. In our case combining it with the `pd.isna()` function gives us a Series of boolean corresponding to the values that are not NaN. We can use this to create an **"answered"** column.

In [6]:
df["answered"] = ~(df.accepted_answer == 0)
df.head()

,question_id,title,tags,creation_date,accepted_answer,favorite_count,answer_count,comment_count,view_count,answered
0,44919655,How to add an onclick method on dynamically cr...,javascript|jquery,2017-07-05 07:29:18.98 UTC,0,0,5,11,52,False
1,45962870,Html clone in Javascript,javascript|jquery|html,2017-08-30 14:16:55.68 UTC,45964230,2,5,11,106,True
2,47409716,'number' is not a valid type for an input tag ...,asp.net|html5|iis|webforms,2017-11-21 09:36:17.393 UTC,47515741,0,7,14,227,True
3,42415666,Error:Cause: unable to find valid certificatio...,android|android-studio,2017-02-23 12:22:43.983 UTC,0,1,7,12,17553,False
4,45001657,Codeigniter User's Data,codeigniter|view|details,2017-07-09 22:29:15.42 UTC,0,0,7,15,721,False


In [7]:
df["answered"] = df["answered"].astype(int) 
df.answered[:10]

0    0
1    1
2    1
3    0
4    0
5    0
6    1
7    0
8    0
9    1
Name: answered, dtype: int64

In [8]:
df.head()

,question_id,title,tags,creation_date,accepted_answer,favorite_count,answer_count,comment_count,view_count,answered
0,44919655,How to add an onclick method on dynamically cr...,javascript|jquery,2017-07-05 07:29:18.98 UTC,0,0,5,11,52,0
1,45962870,Html clone in Javascript,javascript|jquery|html,2017-08-30 14:16:55.68 UTC,45964230,2,5,11,106,1
2,47409716,'number' is not a valid type for an input tag ...,asp.net|html5|iis|webforms,2017-11-21 09:36:17.393 UTC,47515741,0,7,14,227,1
3,42415666,Error:Cause: unable to find valid certificatio...,android|android-studio,2017-02-23 12:22:43.983 UTC,0,1,7,12,17553,0
4,45001657,Codeigniter User's Data,codeigniter|view|details,2017-07-09 22:29:15.42 UTC,0,0,7,15,721,0


## Working with strings

Were we operating inside the boundaries of vanilla Python we would probably write a list comprehension using the `.split()` available for strings. Pandas let us grow beyond the limit imposed by the for loop offering us a host of vectorized string operations, accessible by calling the `.str` method on a Series, also capable of working with NaN and missing data in general without breaking.

### Fixing *creation_date*

Fearing that **creation_date** is not a simple datetime object we decide to check the types contained inside the Series.

In [9]:
type(df.creation_date[0])

str

As we suspected it is indeed a string. One way to easily extract the date part is to split the string in three parts using the split methods.

In [10]:
dates = df.creation_date.str.split(" ").apply(lambda x: x[0])
dates[:10]

0    2017-07-05
1    2017-08-30
2    2017-11-21
3    2017-02-23
4    2017-07-09
5    2017-05-29
6    2017-08-31
7    2017-05-23
8    2017-11-11
9    2017-03-29
Name: creation_date, dtype: object

In [11]:
df["creation_date"] = dates
df.head()

,question_id,title,tags,creation_date,accepted_answer,favorite_count,answer_count,comment_count,view_count,answered
0,44919655,How to add an onclick method on dynamically cr...,javascript|jquery,2017-07-05,0,0,5,11,52,0
1,45962870,Html clone in Javascript,javascript|jquery|html,2017-08-30,45964230,2,5,11,106,1
2,47409716,'number' is not a valid type for an input tag ...,asp.net|html5|iis|webforms,2017-11-21,47515741,0,7,14,227,1
3,42415666,Error:Cause: unable to find valid certificatio...,android|android-studio,2017-02-23,0,1,7,12,17553,0
4,45001657,Codeigniter User's Data,codeigniter|view|details,2017-07-09,0,0,7,15,721,0


### Cleaning *tags*

Let us now assume that we would like to limit our analysis to a subset of the tags available. Currently all values inside the **"tags"** are unusable due to their being just a single string containing all the succulent data only separated by |.

[Strings splitting in Pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html#pandas-series-str-split)

---

Normally in vanilla Python we would do something like:

```python
df["tags"] = [s.split("|") for s in df.tags]
```

This would break since we do have NaN inside the Series. The error that is thrown with this code will mention that the `.split()` method does not work on float since NaN are technically float, however do not be fooled, a Series, like a NumPy Array only contains objects of same types.

In [12]:
df.tags[:10]

0                        javascript|jquery
1                   javascript|jquery|html
2               asp.net|html5|iis|webforms
3                   android|android-studio
4                 codeigniter|view|details
5                               php|arrays
6    c|assembly|c11|function-call|noreturn
7                               javascript
8       python|tensorflow|object-detection
9                           php|html|mysql
Name: tags, dtype: object

In [13]:
split_tags = df.tags.str.split("|")
split_tags[:10]

0                           [javascript, jquery]
1                     [javascript, jquery, html]
2                [asp.net, html5, iis, webforms]
3                      [android, android-studio]
4                   [codeigniter, view, details]
5                                  [php, arrays]
6    [c, assembly, c11, function-call, noreturn]
7                                   [javascript]
8         [python, tensorflow, object-detection]
9                             [php, html, mysql]
Name: tags, dtype: object

We now overwrite the old unusable tags

In [14]:
df.tags = split_tags
df.head()

,question_id,title,tags,creation_date,accepted_answer,favorite_count,answer_count,comment_count,view_count,answered
0,44919655,How to add an onclick method on dynamically cr...,"[javascript, jquery]",2017-07-05,0,0,5,11,52,0
1,45962870,Html clone in Javascript,"[javascript, jquery, html]",2017-08-30,45964230,2,5,11,106,1
2,47409716,'number' is not a valid type for an input tag ...,"[asp.net, html5, iis, webforms]",2017-11-21,47515741,0,7,14,227,1
3,42415666,Error:Cause: unable to find valid certificatio...,"[android, android-studio]",2017-02-23,0,1,7,12,17553,0
4,45001657,Codeigniter User's Data,"[codeigniter, view, details]",2017-07-09,0,0,7,15,721,0


## Python Questions

In [15]:
filter_python_tags = lambda x: "python" in x
py_tags = df.tags.fillna(value="notag").apply(filter_python_tags)
python_questions = df[py_tags]
python_questions.head()

,question_id,title,tags,creation_date,accepted_answer,favorite_count,answer_count,comment_count,view_count,answered
8,47237388,Nothing is being detected in Tensorflow Object...,"[python, tensorflow, object-detection]",2017-11-11,0,5,5,12,319,0
10,46373895,How to open a huge excel file efficiently,"[java, c#, python, c++, excel]",2017-09-22,46434061,11,12,30,1929,1
36,44900912,Numpy's float32 and float comparisons,"[python, python-3.x, numpy, floating-point]",2017-07-04,44922312,0,5,13,256,1
44,43516792,Call Python or Lua from C++ to evaluate an exp...,"[python, c++, lua, lazy-evaluation]",2017-04-20,43745825,2,12,12,689,1
54,45224792,Avoid generating duplicate values from random,"[python, random]",2017-07-20,0,0,5,15,84,0


In [16]:
pq_grouped = python_questions.groupby(python_questions.creation_date)
pq = pd.DataFrame()
pq["daily_questions"] = pq_grouped.question_id.count()
pq.head()

,daily_questions
creation_date,
2017-01-30,577
2017-01-31,659
2017-02-01,703
2017-02-02,673
2017-02-03,612


## Visualization

In [18]:
import altair